This notebook will train segmentation models using the MMsegmentation package

## Install MMSegmentation and other packages, connect to gdrive
This step may take several minutes. 

We use PyTorch 1.10 and CUDA 11.1 for this tutorial. You may install other versions by change the version number in pip install command. 

In [ ]:
# Check nvcc version
!nvcc -V
# Check GCC version
!gcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0
gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
Copyright (C) 2019 Free Software Foundation, Inc.
This is free software; see the source for copying conditions.  There is NO
warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [ ]:
# Install PyTorch
conda install pytorch==1.12.0 torchvision==0.13.0 torchaudio==0.12.0 cudatoolkit=11.3 -c pytorch
# Install mim
!pip install -U openmim
# Install mmengine
# !mim install mmengine - #this version 0.7.2 seems to cause an error, possibly already resolved
!mim install 'mmengine==0.7'
# Install MMCV
!mim install 'mmcv >= 2.0.0rc1'

/bin/bash: conda: command not found
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu118/torch2.0.0/index.html
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/setuptools/command/install.py:34: SetuptoolsDeprecationWarning: setup.py install is deprecated. Use build and pip and other standards-based tools.
  warnings.warn(
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
#clone mmsegmentation repo
%cd /content
!rm -rf mmsegmentation
!git clone --branch dev-1.x --single-branch https://github.com/open-mmlab/mmsegmentation.git
# !git clone https://github.com/open-mmlab/mmsegmentation.git
# mmseg_folder = ~/content/gdrive/MyDrive/CaresAI/Code/segmentation
%cd mmsegmentation
!pip install -e .

/content
Cloning into 'mmsegmentation'...
remote: Enumerating objects: 13109, done.
remote: Counting objects: 100% (313/313), done.
remote: Compressing objects: 100% (239/239), done.
remote: Total 13109 (delta 102), reused 177 (delta 69), pack-reused 12796
Receiving objects: 100% (13109/13109), 18.95 MiB | 6.99 MiB/s, done.
Resolving deltas: 100% (9633/9633), done.
/content/mmsegmentation
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Obtaining file:///content/mmsegmentation
  Preparing metadata (setup.py) ... done
  Running setup.py develop for mmsegmentation


In [ ]:
# #clone mmengine repo
# %cd /content
# !rm -rf mmengine
# # !git clone --branch 1.x --single-branch https://github.com/open-mmlab/mmsegmentation.git
# !git clone https://github.com/open-mmlab/mmengine.git
# # mmseg_folder = ~/content/gdrive/MyDrive/CaresAI/Code/segmentation
# %cd mmengine
# !pip install -e .
# # %cd /content/gdrive

We run an basic inference model to check the enviroment is configured corectly

Expected result:
After running demo cell, a segmentation prediction image is output to result.jpg

In [ ]:
#checking env is correct
!mim download mmsegmentation --config pspnet_r50-d8_4xb2-40k_cityscapes-512x1024 --dest .

processing pspnet_r50-d8_4xb2-40k_cityscapes-512x1024...
downloading ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.1/187.1 MiB 77.4 MB/s eta 0:00:00
Successfully downloaded pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth to /content/mmsegmentation
Successfully dumped pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py to /content/mmsegmentation


In [ ]:
!python3 demo/image_demo.py demo/demo.png configs/pspnet/pspnet_r50-d8_4xb2-40k_cityscapes-512x1024.py pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth --device cuda:0 --out-file result.jpg

/content/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(
Loads checkpoint by local backend from path: pspnet_r50-d8_512x1024_40k_cityscapes_20200605_003338-2966598c.pth
04/19 16:42:19 - mmengine - WARNING - `Visualizer` backend is not initialized because save_dir is None.


Import needed packages:

In [ ]:
# Pytorch installation
import torch, torchvision

#import mmseg
# import mmseg

# # # MMSegmentation installation
# import mmcv
# import mmengine

#other
import numpy as np
import os
import typing
from typing import Callable
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
import pandas as pd
import cv2
from google.colab.patches import cv2_imshow
from PIL import Image
import seaborn as sns

In [ ]:
print(f'torch version: {torch.__version__}')
print(f'check if cuda is available: {torch.cuda.is_available()}')
print(f'mmcv_version: {mmcv.__version__}')
print(f'mmengine_version: {mmengine.__version__}')
print(f'mmsegmentation_version: {mmseg.__version__}')

torch version: 2.0.0+cu118
check if cuda is available True
mmcv_version: 2.0.0
mmengine_version: 0.7.0
mmsegmentation_version: 1.0.0


## Creating train/val splits

https://bcsegmentation.grand-challenge.org/Baseline/

Slides from these institutes were not used in training the final segmentation model on the core set. They were used as an unseen testing set to report accuracy: OL, LL, E2, EW, GM, and S3.  <- Please use this testing set to reproduce the accuracy results from the table above.

In [ ]:
#set paths to images and masks
data_root = '/content/gdrive/MyDrive/BCSS_dataset'

use_patched = True
use_3d_masks = False
if (use_patched == True) and (use_3d_masks==False):
  img_dir_path = os.path.join(data_root,'patched_images_masks_2d_6_class','images')
  ann_dir_path = os.path.join(data_root,'patched_images_masks_2d_6_class','masks')
elif (use_patched == False) and (use_3d_masks==True):
  img_dir_path = img_dir_path = prep_dataset_obj.orig_images_folder_path
  ann_dir_path = os.path.join(data_root,'new_masks_3d_6_class')
elif (use_patched == False) and (use_3d_masks==False):
  img_dir_path = os.path.join(data_root,'images')
  ann_dir_path = os.path.join(data_root,'new_masks_2d_6_class')
print(img_dir_path)
print(ann_dir_path)

/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/images
/content/gdrive/MyDrive/BCSS_dataset/patched_images_masks_2d_6_class/masks


In [ ]:
split_dir = 'splits'
split_path = os.path.join(data_root, split_dir)
if not os.path.exists(split_path):
  os.makedirs(split_path)

#get list of filenames
filename_list = [os.path.splitext(filename)[0] for filename in os.listdir(ann_dir_path)]
train_labs = ['E9', 'GI', 'HN', 'D8', 'BH', 'C8', 'A7', 'A8', 'AC', 'AN', 'AO', 'AQ', 'AR', 'A1', 'A2']
train_files = []
val_files = []
test_files = []
val_labs = ['OL', 'LL', 'EW', 'GM','S3']
test_lab = 'E2' #randomly selected from validation/test sets
train_counter = dict.fromkeys(train_labs, 0)
val_counter = dict.fromkeys(val_labs, 0) #start a counter to count files

test_counter = 0

#loop through filenames and assign into train/val/test set as needed
for filename in filename_list:
  lab_name = filename.split('-')[1]
  if lab_name in val_labs:
    val_counter[lab_name] = val_counter[lab_name] + 1
    val_files.append(filename)
  elif lab_name in test_lab:
    test_counter = test_counter+1
    test_files.append(filename)
  else:
    train_files.append(filename)
    train_counter[lab_name] = train_counter[lab_name] + 1

train_files = sorted(train_files)
val_files = sorted(val_files)

num_train_imgs = len(train_files)
num_val_imgs = len(val_files)
num_test_imgs = len(test_files)
total_imgs = num_train_imgs + num_val_imgs + num_test_imgs
print(f"num train files: {num_train_imgs} , % of total: {(num_train_imgs/total_imgs):.3f}")
print(f"num val files: {num_val_imgs} , % of total: {(num_val_imgs/total_imgs):.3f}")
print(f"num test files: {num_test_imgs} , % of total: {(num_test_imgs/total_imgs):.3f}")

print('train counts:')
print(train_counter)

print('val_counts')
print(val_counter)

with open(os.path.join(data_root, split_dir, 'train.txt'), 'w') as f:
  f.writelines(line + '\n' for line in train_files)

with open(os.path.join(data_root, split_dir, 'val.txt'), 'w') as f:
  f.writelines(line + '\n' for line in val_files)

# print(train_labs)

num train files: 3318 , % of total: 0.690
num val files: 974 , % of total: 0.202
num test files: 520 , % of total: 0.108
train counts:
{'E9': 66, 'GI': 15, 'HN': 9, 'D8': 621, 'BH': 603, 'C8': 261, 'A7': 273, 'A8': 88, 'AC': 132, 'AN': 150, 'AO': 274, 'AQ': 28, 'AR': 204, 'A1': 55, 'A2': 539}
val_counts
{'OL': 188, 'LL': 62, 'EW': 441, 'GM': 249, 'S3': 34}
['E9', 'GI', 'HN', 'D8', 'BH', 'C8', 'A7', 'A8', 'AC', 'AN', 'AO', 'AQ', 'AR', 'A1', 'A2']


In [ ]:
train_str = str(train_counter)
train_str = train_str.replace("'","")
print(train_str)

{E9: 66, GI: 15, HN: 9, D8: 621, BH: 603, C8: 261, A7: 273, A8: 88, AC: 132, AN: 150, AO: 274, AQ: 28, AR: 204, A1: 55, A2: 539}


## Train a semantic segmentation model on a new dataset

To train on a customized dataset, the following steps are necessary. 
1. Add a new dataset class. 
2. Create a config file accordingly. 
3. Perform training and evaluation. 

### Add a new dataset

Datasets in MMSegmentation require image and semantic segmentation maps to be placed in folders with the same prefix. To support a new dataset, we may need to modify the original file structure. 

In this tutorial, we give an example of converting the dataset. You may refer to [docs](https://github.com/open-mmlab/mmsegmentation/blob/master/docs/en/tutorials/customize_datasets.md#customize-datasets-by-reorganizing-data) for details about dataset reorganization. 

We use [Stanford Background Dataset](http://dags.stanford.edu/projects/scenedataset.html) as an example. The dataset contains 715 images chosen from existing public datasets [LabelMe](http://labelme.csail.mit.edu), [MSRC](http://research.microsoft.com/en-us/projects/objectclassrecognition), [PASCAL VOC](http://pascallin.ecs.soton.ac.uk/challenges/VOC) and [Geometric Context](http://www.cs.illinois.edu/homes/dhoiem/). Images from these datasets are mainly outdoor scenes, each containing approximately 320-by-240 pixels. 
In this tutorial, we use the region annotations as labels. There are 8 classes in total, i.e. sky, tree, road, grass, water, building, mountain, and foreground object. 

In [ ]:
# download and unzip
# !wget http://dags.stanford.edu/data/iccv09Data.tar.gz -O stanford_background.tar.gz
# !tar xf stanford_background.tar.gz

## Creating dataset object

After downloading the data, we need to implement `load_annotations` function in the new dataset class `StanfordBackgroundDataset`.

In [ ]:
%%script false --no-raise-error
#example
from mmseg.datasets.builder import DATASETS
from mmseg.datasets.custom import CustomDataset

@DATASETS.register_module()
class StanfordBackgroundDataset(CustomDataset):
  CLASSES = classes
  PALETTE = palette
  def __init__(self, split, **kwargs):
    super().__init__(img_suffix='.jpg', seg_map_suffix='.png', 
                     split=split, **kwargs)
    assert osp.exists(self.img_dir) and self.split is not None

    

https://github.com/open-mmlab/mmsegmentation/blob/104429be008d84cd4ca7f162e2fd04f93c77e8ad/docs/en/tutorials/customize_datasets.md

https://github.com/open-mmlab/mmsegmentation/blob/5b438f45b6134368d2c1717446b954a1457743f0/mmseg/datasets/custom.py

In [ ]:
class_names_new = ['outside_roi','tumor','stroma','inflammatory','necrosis','other']
classes_new_int = [0,1,2,3,4,5]
# convert dataset annotation to semantic segmentation map
palette = sns.color_palette('tab20', len(classes_new_int))                                            
palette_array = np.asarray(palette)*255
# roll array so orange is tumour 
palette_array = np.roll(palette_array, -2,axis=0)
palette_array #shows palette

array([[255., 127.,  14.],
       [255., 187., 120.],
       [ 44., 160.,  44.],
       [152., 223., 138.],
       [ 31., 119., 180.],
       [174., 199., 232.]])

In [ ]:
#cancer dataset
from mmseg.registry import DATASETS
from mmseg.datasets import BaseSegDataset

@DATASETS.register_module()
class CancerDataset(BaseSegDataset):
  METAINFO = dict(classes = class_names_new, palette = palette_array)
  def __init__(self, **kwargs):
    super().__init__(img_suffix='.png', 
                     seg_map_suffix='.png', 
                     **kwargs)
  
  # def evaluate(self,
  #               results,
  #               metric='mDice',
  #               logger=None,
  #               gt_seg_maps=None,
  #               **kwargs):
    
  #   with torch.no_grad():
  #     metric='mDice'
  #     super().evaluate(results,
  #                     metric,
  #                     logger,
  #                     gt_seg_maps)

# cancerDataset_obj = CancerDataset()

### Create a config file

We now need to create a config file which defines how the data is loaded and the parameters for training and validating the model

In [ ]:
%cd /content

/content


In [ ]:
#download the checkpoints for the model of interest
current_model_path = os.path.join('mmsegmentation/checkpoints','current_model')
!rm $current_model_path -r

!wget https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_512x512_160k_ade20k/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth -P $current_model_path
checkpoint_path = os.path.join(current_model_path,os.listdir(current_model_path)[0])
print(f'checkpoint path: {checkpoint_path}')

rm: cannot remove 'mmsegmentation/checkpoints/current_model': No such file or directory
--2023-04-19 16:42:33--  https://download.openmmlab.com/mmsegmentation/v0.5/segformer/segformer_mit-b5_512x512_160k_ade20k/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth
Resolving download.openmmlab.com (download.openmmlab.com)... 47.246.48.204, 47.246.48.205, 47.246.48.206, ...
Connecting to download.openmmlab.com (download.openmmlab.com)|47.246.48.204|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 328404259 (313M) [application/octet-stream]
Saving to: ‘mmsegmentation/checkpoints/current_model/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth’

segformer_mit-b5_51 100%[===================>] 313.19M  99.0MB/s    in 3.2s    

2023-04-19 16:42:37 (99.0 MB/s) - ‘mmsegmentation/checkpoints/current_model/segformer_mit-b5_512x512_160k_ade20k_20210726_145235-94cedf59.pth’ saved [328404259/328404259]

checkpoint path: mmsegmentation/checkpoints/c

In [ ]:
#import relevant config file for model of interest
from mmengine import Config
cfg_orig = Config.fromfile('/content/mmsegmentation/configs/segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py')
cfg = Config.fromfile('/content/mmsegmentation/configs/segformer/segformer_mit-b5_8xb2-160k_ade20k-512x512.py')

Example config fle from tutorial:

https://github.com/open-mmlab/mmsegmentation/blob/104429be008d84cd4ca7f162e2fd04f93c77e8ad/docs/en/tutorials/data_pipeline.md

Config file for cancer dataset:

N.b if changing options in the config file, it seems necessary to reload the config file by rerunning the above command 
"cfg = Config.fromfile("

Create config file to control data loading and model training process:

In [ ]:
# Since the given config is used to train PSPNet on the cityscapes dataset, we need to modify it accordingly for our new dataset.
# Load the pretrained weights
cfg.load_from = checkpoint_path

#set batch normalisation settings
# Since we use only one GPU, BN is used instead of SyncBN . 
# cfg.model.backbone.norm_cfg is kept as is , causes an error if set to BN
cfg.norm_cfg = dict(type='BN', requires_grad=True)
cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg

#define paths for imgs and annotations
data_prefix_dict = dict(img_path=img_dir_path, seg_map_path=ann_dir_path)

#set segmentation index to ignore - class 0 = outside ROI
cfg.model.decode_head.ignore_index = 0
cfg_orig.model.decode_head.loss_decode.avg_non_ignore = True #we don't want to take into account ignored seg indexes when calculating losses

# modify num classes of the model in decode/auxiliary head
cfg.model.decode_head.num_classes = 6 - 1 #6 total classes minus ignored class of 0
# cfg.model.auxiliary_head.num_classes = 6

#set model.pretrained to None as we have already loaded the checkpoints
cfg.model.pretrained = None

# Modify dataset type and path
cfg.dataset_type = 'CancerDataset'
cfg.data_root = data_root

#set some hyperparameter values for training
BATCH_SIZE = 4
NUM_WORKERS = 1
USE_PERS_WORKERS = False

#set image augmentation hyperparameters
CROP_SIZE = (758, 758)
RESIZE_SIZE = (256,256)

#define the pipeline for loading training images and masks
cfg.train_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='LoadAnnotations'),
    dict(type='RandomCrop', crop_size=CROP_SIZE),
    # dict(type='PhotoMetricDistortion'),
    dict(type='Resize', scale=RESIZE_SIZE),#, ratio_range=(0.5, 2.0), keep_ratio=True),
    dict(type='RandomFlip', prob=0.5),
    dict(type='PackSegInputs')
]

#define the pipeline for loading validatrion/test images and masks
cfg.test_pipeline = [
    dict(type='LoadImageFromFile'),
    dict(type='Resize', scale=RESIZE_SIZE),
    # add loading annotation after ``Resize`` because ground truth
    # does not need to do resize data transform
    dict(type='LoadAnnotations'),
    dict(type='PackSegInputs')
]

#define variables for loading training images and masks
cfg.train_dataloader.dataset.pipeline = cfg.train_pipeline
cfg.train_dataloader.dataset.type = cfg.dataset_type
cfg.train_dataloader.dataset.data_root = cfg.data_root
cfg.train_dataloader.dataset.data_prefix = data_prefix_dict
cfg.train_dataloader.dataset.ann_file = 'splits/train.txt'
cfg.train_dataloader.batch_size = BATCH_SIZE
cfg.train_dataloader.num_workers= NUM_WORKERS
cfg.train_dataloader.persistent_workers=USE_PERS_WORKERS
cfg.train_dataloader.sampler = dict(type='DefaultSampler', shuffle=True)

#define variables for loading validation images and masks
cfg.val_dataloader.dataset.pipeline = cfg.test_pipeline
cfg.val_dataloader.dataset.type = cfg.dataset_type
cfg.val_dataloader.dataset.data_root = cfg.data_root
cfg.val_dataloader.dataset.data_prefix = data_prefix_dict
cfg.val_dataloader.dataset.ann_file = 'splits/val.txt'
# cfg.val_dataloader.batch_size = 1
cfg.val_dataloader.num_workers=NUM_WORKERS
cfg.val_dataloader.persistent_workers=USE_PERS_WORKERS
cfg.val_dataloader.sampler = dict(type='DefaultSampler', shuffle=True)

#define variables for loading training images and masks
cfg.test_dataloader = cfg.val_dataloader

# Set up working dir to save files and logs.
cfg.work_dir = '/content/gdrive/MyDrive/BCSS_dataset/logs'

# define learning rate scheduler and others
param_scheduler = [
    dict(
        type='LinearLR', start_factor=1e-05, by_epoch=False, begin=0,
        end=500), #was 1500
    dict(
        type='PolyLR',
        eta_min=0.0,
        power=1.0,
        begin=500,#was 1500
        end=160000,
        by_epoch=False)]

#define model training parameters
cfg.train_cfg.max_iters = 1000 #max number of iterations
cfg.train_cfg.val_interval = 500 #number of iterations to check model against validation dataset
cfg.default_hooks.logger.interval = 100 #number of iteratios to print status
cfg.default_hooks.checkpoint.interval = 1000 #how often to save checkpoint of the model 
cfg.log_level = 'INFO' #logging level during training
cfg.log_processor = dict(by_epoch=False)
# cfg.custom_hooks=[dict()]

# Set seed to facilitate reproducing the result
cfg['randomness'] = dict(seed=0)

# set metrics to use during evaluation
cfg.val_evaluator = dict(type='IoUMetric',ignore_index=0,iou_metrics=['mDice','mIoU'])
cfg.test_evaluator = dict(type='IoUMetric', iou_metrics=['mDice'])

# check final config
print(f'Config:\n{cfg.pretty_text}')

Config:
norm_cfg = dict(type='BN', requires_grad=True)
data_preprocessor = dict(
    type='SegDataPreProcessor',
    mean=[123.675, 116.28, 103.53],
    std=[58.395, 57.12, 57.375],
    bgr_to_rgb=True,
    pad_val=0,
    seg_pad_val=255,
    size=(512, 512))
model = dict(
    type='EncoderDecoder',
    data_preprocessor=dict(
        type='SegDataPreProcessor',
        mean=[123.675, 116.28, 103.53],
        std=[58.395, 57.12, 57.375],
        bgr_to_rgb=True,
        pad_val=0,
        seg_pad_val=255,
        size=(512, 512)),
    pretrained=None,
    backbone=dict(
        type='MixVisionTransformer',
        in_channels=3,
        embed_dims=64,
        num_stages=4,
        num_layers=[3, 6, 40, 3],
        num_heads=[1, 2, 5, 8],
        patch_sizes=[7, 3, 3, 3],
        sr_ratios=[8, 4, 2, 1],
        out_indices=(0, 1, 2, 3),
        mlp_ratio=4,
        qkv_bias=True,
        drop_rate=0.0,
        attn_drop_rate=0.0,
        drop_path_rate=0.1),
    decode_head=dict(
      

Below is config file from previous iteration , not currently used but can be checked for inspiration on which things to change to improve model performance

In [ ]:
# #modify config file for cancer training


# #define weights for each class
# class_weights = [0,   #outside_roi
#                  1.2, #tumor
#                  1, #stroma
#                  1, #inflammatory
#                  1, #necrosis
#                  0.95] #other
# try:
#   assert(len(class_weights)==len(classes_new_int))
# except:
#   raise ValueError('len class weights does not match num of classes')


# cfg.norm_cfg = dict(type='BN', requires_grad=True) # Since we use only one GPU, BN is used instead of SyncBN
# # cfg.optimizer = dict(type='AdamW', lr=0.001, weight_decay=0.01)

# #customise training 
# #https://github.com/open-mmlab/mmsegmentation/blob/4eaa8e69191cc293b64dafe47f1f88a7d468c93c/docs/en/tutorials/customize_runtime.md
# cfg.lr_config = dict(
#     policy='poly',
#     warmup='linear',
#     warmup_iters=10,
#     warmup_ratio=1e-06,
#     power=1.0,
#     min_lr=0.0,
#     by_epoch=False)

# # cfg.momentum_config = dict(
# #     policy='cyclic',
# #     target_ratio=(0.85 / 0.95, 1),
# #     cyclic_times=1,
# #     step_ratio_up=0.4,
# # )    

# #define loss type
# # loss_type = dict(type='DiceLoss',
# #                  class_weight=class_weights,
# #                  ignore_index=0,
# #                  avg_non_ignore=True)

# # loss_type = dict(type='FocalLoss',
# #                  class_weight=class_weights,
# #                  loss_weight=1.0,
# #                  loss_name='loss_focal')

# # loss_type = dict(type='LovaszLoss',
# #                  loss_type = 'multi_class',
# #                  class_weight=class_weights,
# #                  reduction='none') # https://github.com/open-mmlab/mmsegmentation/tree/a82ebad0f6aa3cc88452d7442730d60f27c41153/mmseg/models/losses

# #set backbone variables
# # cfg.model.backbone.norm_cfg = cfg.norm_cfg

# #set decode head variables
# cfg.model.decode_head.norm_cfg = cfg.norm_cfg
# # cfg.model.decode_head.ignore_index = 0 #ignore 0 pixels as outside ROI
# cfg.model.decode_head.num_classes = len(classes_new_int)
# # cfg.model.decode_head.sampler = dict(type='OHEMPixelSampler', thresh=0.7, min_kept=100000)
# # cfg.model.decode_head.loss_decode = loss_type
# cfg.model.decode_head.loss_decode.class_weight = class_weights
# cfg.model.decode_head.loss_decode.avg_non_ignore = True
# cfg.model.decode_head.downsample_label_ratio = 0 #trying to resolve mismatch in spatial dimensions

# #set auxillary head variables
# # cfg.model.auxiliary_head.norm_cfg = cfg.norm_cfg
# # cfg.model.auxiliary_head.num_classes = len(classes_new_int)
# # cfg.model.auxiliary_head.ignore_index = 0
# # # cfg.model.decode_head.loss_decode = loss_type
# # cfg.model.auxiliary_head.loss_decode.class_weight = class_weights

# # Modify dataset type and path
# cfg.dataset_type = 'CancerDataset'
# cfg.data_root = data_root

# cfg.data.samples_per_gpu = 2
# cfg.data.workers_per_gpu=1

# cfg.img_norm_cfg = dict({'mean': [123.675, 116.28, 103.53], 
#                         'std': [58.395, 57.12, 57.375], 
#                         'to_rgb': True})

# #https://github.com/open-mmlab/mmsegmentation/blob/2d66179630035097dcae08ee958f60d4b5a7fcae/mmseg/datasets/pipelines/transforms.py
# cfg.crop_size = (512, 512)
# cfg.train_pipeline = [
#     dict(type='LoadImageFromFile',imdecode_backend='pillow'),
#     dict(type='LoadAnnotations',imdecode_backend='pillow'),
#     dict(type='RandomCrop', crop_size=cfg.crop_size),#, cat_max_ratio=0.75),
#     # dict(type='Resize', img_scale=(512, 512)),# ratio_range=(0.5, 2.0)),d
#     dict(type='RandomFlip', flip_ratio=0.1),
#     dict(type='PhotoMetricDistortion'),
#     dict(type='Normalize', **cfg.img_norm_cfg),
#     # dict(type='Pad', size=cfg.crop_size, pad_val=0, seg_pad_val=255),
#     dict(type='DefaultFormatBundle'),
#     dict(type='Collect', keys=['img', 'gt_semantic_seg']),
# ]

# test_pipeline = [
#     dict(type='LoadImageFromFile'),
#     dict(
#         type='MultiScaleFlipAug',
#         img_scale=(512, 512),
#         # img_ratios=[0.5, 0.75, 1.0, 1.25, 1.5, 1.75],
#         flip=False,
#         transforms=[
#             dict(type='Resize', keep_ratio=True),
#             # resize image to multiple of 32, improve SegFormer by 0.5-1.0 mIoU.
#             dict(type='ResizeToMultiple', size_divisor=32),
#             dict(type='RandomFlip'),
#             dict(type='Normalize', **cfg.img_norm_cfg),
#             dict(type='ImageToTensor', keys=['img']),
#             dict(type='Collect', keys=['img']),
#         ])
# ]


# cfg.data.train.type = cfg.dataset_type
# cfg.data.train.data_root = cfg.data_root
# cfg.data.train.img_dir = img_dir_path
# cfg.data.train.ann_dir = ann_dir_path
# cfg.data.train.pipeline = cfg.train_pipeline
# cfg.data.train.split = 'splits/train.txt'

# cfg.data.val.type = cfg.dataset_type
# cfg.data.val.data_root = cfg.data_root
# cfg.data.val.img_dir = img_dir_path
# cfg.data.val.ann_dir = ann_dir_path
# cfg.data.val.pipeline = cfg.test_pipeline
# cfg.data.val.split = 'splits/val.txt'

# cfg.data.test.type = cfg.dataset_type
# cfg.data.test.data_root = cfg.data_root
# cfg.data.test.img_dir = img_dir_path
# cfg.data.test.ann_dir = ann_dir_path
# cfg.data.test.pipeline = cfg.test_pipeline
# cfg.data.test.split = 'splits/val.txt'

# # cfg.model.decode_head.train_cfg = cfg.train_pipeline
# # cfg.test_cfg = dict(mode='whole')
# #cfg.test_cfg = dict(mode='slide', crop_size=(768, 768), stride=(256, 256))

# # We can still use the pre-trained Mask RCNN model though we do not need to
# # use the mask branch
# # cfg.load_from = '/content/mmsegmentation/checkpoints/segformer_mit-b3_512x512_160k_ade20k.py'

# # Set up working dir to save files and logs.
# !rm $cfg.work_dir -r
# cfg.work_dir = os.path.join('/content/mmsegmentation/checkpoints/current_model','working_dir')

# full_epochs = 100
# cfg.runner.max_iters = full_epochs
# cfg.log_config.interval = 10 #interval to display training status
# cfg.evaluation.interval = full_epochs
# cfg.checkpoint_config.interval = 20

# # Set seed to facitate reproducing the result
# cfg.seed = 0
# set_random_seed(0, deterministic=False)
# cfg.gpu_ids = range(1)
# cfg.device = get_device()

# # Let's have a look at the final config used for training
# print(f'Config:\n{cfg.pretty_text}')

### Train and Evaluation

Now we load the config file into a runner format. The runner handles all the backend model training, which processes to run a twhich times etc

In [ ]:
from mmengine.runner import Runner

runner = Runner.from_cfg(cfg)


04/19 16:42:40 - mmengine - INFO - 
------------------------------------------------------------
System environment:
    sys.platform: linux
    Python: 3.9.16 (main, Dec  7 2022, 01:11:51) [GCC 9.4.0]
    CUDA available: True
    numpy_random_seed: 0
    GPU 0: Tesla T4
    CUDA_HOME: /usr/local/cuda
    NVCC: Cuda compilation tools, release 11.8, V11.8.89
    GCC: x86_64-linux-gnu-gcc (Ubuntu 9.4.0-1ubuntu1~20.04.1) 9.4.0
    PyTorch: 2.0.0+cu118
    PyTorch compiling details: PyTorch built with:
  - GCC 9.3
  - C++ Version: 201703
  - Intel(R) oneAPI Math Kernel Library Version 2022.2-Product Build 20220804 for Intel(R) 64 architecture applications
  - Intel(R) MKL-DNN v2.7.3 (Git Hash 6dbeffbae1f23cbbeae17adb7b5b13f1f37c080e)
  - OpenMP 201511 (a.k.a. OpenMP 4.5)
  - LAPACK is enabled (usually provided by MKL)
  - NNPACK is enabled
  - CPU capability usage: AVX2
  - CUDA Runtime 11.8
  - NVCC architecture flags: -gencode;arch=compute_37,code=sm_37;-gencode;arch=compute_50,code=sm_5

/content/mmsegmentation/mmseg/models/builder.py:36: UserWarning: ``build_loss`` would be deprecated soon, please use ``mmseg.registry.MODELS.build()`` 
  warnings.warn('``build_loss`` would be deprecated soon, please use '
/content/mmsegmentation/mmseg/models/losses/cross_entropy_loss.py:235: UserWarning: Default ``avg_non_ignore`` is False, if you would like to ignore the certain label and average loss over non-ignore labels, which is the same with PyTorch official cross_entropy, set ``avg_non_ignore=True``.
  warnings.warn(


04/19 16:42:49 - mmengine - INFO - Distributed training is not used, all SyncBatchNorm (SyncBN) layers in the model will be automatically reverted to BatchNormXd layers if they are used.
04/19 16:42:49 - mmengine - INFO - Hooks will be executed in the following order:
before_run:
(VERY_HIGH   ) RuntimeInfoHook                    
(BELOW_NORMAL) LoggerHook                         
 -------------------- 
before_train:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(VERY_LOW    ) CheckpointHook                     
 -------------------- 
before_train_epoch:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
(NORMAL      ) DistSamplerSeedHook                
 -------------------- 
before_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                    
(NORMAL      ) IterTimerHook                      
 -------------------- 
after_train_iter:
(VERY_HIGH   ) RuntimeInfoHook                

/content/mmsegmentation/mmseg/engine/hooks/visualization_hook.py:61: UserWarning: The draw is False, it means that the hook for visualization will not take effect. The results will NOT be visualized or stored.
  warnings.warn('The draw is False, it means that the '


In [ ]:
#start model training
runner.train()

04/19 16:42:50 - mmengine - WARNING - The "loop" registry in mmseg did not set import location. Fallback to call `mmseg.utils.register_all_modules` instead.
04/19 16:42:51 - mmengine - WARNING - The "data sampler" registry in mmseg did not set import location. Fallback to call `mmseg.utils.register_all_modules` instead.
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.weight:lr=6e-05
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.weight:weight_decay=0.0
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.weight:decay_mult=0.0
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.bias:lr=6e-05
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.bias:weight_decay=0.0
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.0.norm.bias:decay_mult=0.0
04/19 16:42:51 - mmengine - INFO - paramwise_options -- backbone.layers.0.

RuntimeError: ignored